In [1]:
import sys

import numpy as np

from stixdcpy.quicklook import LightCurves
from stixdcpy.energylut import EnergyLUT
from stixdcpy.housekeeping import Housekeeping
from stixdcpy import ancillarydata as anc
from stixdcpy.sciencedata import SciL1, SciSpectrogram

from matplotlib import pyplot as plt

from pprint import pprint
%matplotlib notebook

In [2]:
#elut=EnergyLUT.request('2021-09-03T00:00:00')

In [3]:
#Print onboard elut and calibration run information
#elut.info()
#elut.data


In [4]:
#emph=anc.Ephemeris.request(start_utc='2021-02-28T00:00:00', end_utc='2021-2-28T01:00:00',steps=2)
#emph.peek()

In [5]:
#emph.data


In [6]:
request_id=	2108010255
sci_data=SciL1.fetch(request_id=request_id)
#Bulk science data request ids (or also called unique ids) can be found on the science data web page at STIX data center

Loading /tmp/solo_L1A_stix-sci-xray-l1-2108010255_20210801T150229-20210801T150529_012533_V01.fits from local archive ...


In [ ]:
data=sci_data.data
#data is a fits file object
data.info()
data[0].header['DATE_BEG']


In [ ]:
sci_data.peek()
#science data quicklook

In [ ]:
data.info()


In [ ]:
#fig=plt.figure()
l1data=data[2].data
#print(l1data['num_energy_groups'])
print(l1data['counts'].shape)
print(l1data['timedel'].shape)
print(l1data['triggers'])
trigger_rates=l1data['triggers'][1:,:]/l1data['timedel'][:-1,None]

tau=9.6e-6
eta=1e-5
live_time=1 - tau*trigger_rates
photo_in_rates=trigger_rates /(1-tau*trigger_rates)

counts=l1data['counts'][1:,:,:,:]/l1data['timedel'][:-1,None, None, None]

print('counts shape:', counts.shape)
#print(counts.shape)
print('triggers shape:', trigger_rates.shape)
out=np.zeros_like(counts)







In [ ]:
print(counts.shape)


DET_GROUPS = [
	[1, 2],
	[6, 7],
	[5, 11],
	[12, 13],
	[14, 15],
	[10, 16],
	[8, 9],
	[3, 4],
	[31, 32],
	[26, 27],
	[22, 28],
	[20, 21],
	[18, 19],
	[17, 23],
	[24, 25],
	[29, 30]
]
for i, dets in enumerate(DET_GROUPS):
    if i>=2:
        break
    #    continue
    d1,d2=dets[0]-1,dets[1]-1
    #trig=
    fig=plt.figure()
    #print(i, d1,d2)
    trig=trigger_rates[:,i]
    count_sum=np.sum(counts[:,d1,:,:]+ counts[:,d2,:,:], axis=(1,2))
    #plt.plot(trig, label=f'trig {i}')
    ratio=count_sum/trig
    #plt.plot(count_sum,label=f'counts of D{d1}+D{d2}')
    plt.plot(count_sum/trig,label=f'trig #{i} / counts of D{d1}+D{d2}')
    #most of the counts are not recorded at the begining due to the energy range 
    plt.legend()
    #break
    
    
    

    #print(i, '#', trig)
    #print(count_sum)
    #print(l1data['timedel'])


sci_data.spectrogram[1:, :]

In [ ]:
x=np.arange(32).reshape((4,8))
x,x[:,None]

In [ ]:
request_id=	2108150003
l4d=SciSpectrogram.fetch(request_id=request_id)

In [ ]:
l4d.data.info()
l4d.peek()

In [ ]:
request_id=2107040009
l1bkg=SciL1.fetch(request_id=request_id)
l1bkg.peek()
